In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPAL import *
from AutoReasoner import *
import dotenv

config = dotenv.dotenv_values(".env")
openai.api_type = 'azure'
openai.api_base = 'https://meta-prompter-az-openai.openai.azure.com'
openai.api_version = '2022-12-01'
openai.api_key = config['OPENAI_API_KEY_ms']

# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables-sample400.tsv', sep='\t')
dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')
# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/training.tsv', sep='\t')
ft = None

In [4]:
import fasttext
ft = fasttext.load_model('cc.en.300.bin')

In [42]:
NNDemo = False
max_demo = 5

template = 'pal-sql-py'
# template = 'formatv1-sql-py'

# gpt_model = 'text-davinci-003'
# gpt_model = 'gpt-3.5-turbo'
gpt_model = 'mp-aoi-codex'

def parallel_codex_func_formatv1(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = Codex_PAL(
                                      f'prompt_template/{template}.json',
                                      dataset.iloc[i]['id'], 
                                      dataset.iloc[i]['utterance'], 
                                      dataset.iloc[i]['context'], 
                                      dataset.iloc[i]['targetValue'],  
                                      base_path='./dataset/WikiTableQuestions/',
                                      demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                )
            codex_prompter.model = gpt_model
            codex_prompter.max_demo = max_demo
            
            # codex_prompter._gen_gpt_prompt()
            codex_prompter._gen_gpt_prompt(NNDemo, ft)
            codex_prompter._get_gpt_prediction()
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            if "model's maximum context length" in str(e):
                return log
            max_retry -= 1
    return log
    
for program in ['sql-py']:
    n_threads = 2
    maxLimit = float('inf')
    maxLimit = 100
    from joblib import Parallel, delayed
    output_result_file = f'./dataset/WikiTableQuestions/results/CodexPAL_{template}_{program}_NNDemo={NNDemo}_results_pristine-unseen-tables_limit{maxLimit}_model{gpt_model}.json'
    logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func_formatv1)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))
    json.dump(logs, open(output_result_file, 'w'), indent=4)
    # evaluate: 
    os.system(f'cd ./dataset/WikiTableQuestions/ && python2 evaluator.py ./results/{output_result_file.split("/")[-1]} && cd ..')


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:19<00:00,  1.40s/it]

Reading dataset from ./tagged/data/training.tagged
Reading dataset from ./tagged/data/pristine-seen-tables.tagged
Reading dataset from ./tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./results/CodexPAL_pal-sql-py_sql-py_NNDemo=False_results_pristine-unseen-tables_limit100_modelmp-aoi-codex.json
Examples: 100
Correct: 64
Accuracy: 0.64
GPT error: 0.0
Uncaught error: 0.12
{u"'NoneType' object ha": [u'nu-3', u'nu-12', u'nu-15', u'nu-22', u'nu-29', u'nu-35', u'nu-44', u'nu-59', u'nu-72', u'nu-80', u'nu-95', u'nu-96']}


In [39]:
NNDemo = False
max_demo = 5
# template = 'original-sql-py-no-intermediate'
template = 'pal-sql-py'
program = 'sql-py'
gpt_model = 'mp-aoi-codex'

def func(i):
    codex_prompter = Codex_PAL(
                                      f'prompt_template/{template}.json',
                                      dataset.iloc[i]['id'], 
                                      dataset.iloc[i]['utterance'], 
                                      dataset.iloc[i]['context'], 
                                      dataset.iloc[i]['targetValue'],  
                                      base_path='./dataset/WikiTableQuestions/',
                                      demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                )
    codex_prompter.model = gpt_model
    codex_prompter.max_demo = max_demo
    # codex_prompter.demo_ids = [0, 1, 2, 3, 6, 8, 11]

    # codex_prompter._gen_gpt_prompt()
    codex_prompter._gen_gpt_prompt()
    codex_prompter._get_gpt_prediction()
    log = codex_prompter._log_dict()
    # print(codex_prompter.prompt)
    return log
a = func(10)

SELECT season, record FROM DF;
def get_wins(s):
    import re
    return int(re.search("(.*?)–", s).group(1))
DF['wins'] = DF.apply(lambda x: get_wins(x['record']), axis=1)

SELECT season, wins FROM DF WHERE wins=11;
